In [1]:
import pandas 
import numpy 
import torch
import torchaudio
import torchvision

import matplotlib
import ultralytics

from ultralytics import YOLO 

import cv2 as cv
import datetime

In [1]:
# !pip install bytetracker
# !pip install supervision==0.1.0


/bin/bash: /home/xcal/anaconda3/envs/Sign/lib/libtinfo.so.6: no version information available (required by /bin/bash)
DEPRECATION: Loading egg at /home/xcal/anaconda3/envs/Sign/lib/python3.11/site-packages/pafprocess_ext-1.0-py3.11-linux-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.3 MB/s eta 0:00:00MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 44.9 MB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 11.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for bytetracker: filename=bytetracker-0.3.2-py3-none-any.whl size=11

In [2]:
import time

In [3]:
model = YOLO("best.pt")

In [4]:
threshold = 0.6 
green = (0,255,0)
red = (0,0,255)
blue = (255,0,0)

In [5]:
cap = cv.VideoCapture(0)

In [6]:
def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv.VideoWriter_fourcc(*'MP4V')
    writer = cv.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

In [7]:
writer = create_video_writer(cap,"output.mp4")

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [8]:
while True:
    # start time to compute the fps
    start = datetime.datetime.now()
    
    ret, frame = cap.read()

    # if there are no more frames to process, break out of the loop
    if not ret:
        break

    # run the YOLO model on the frame
    detections = model(frame)[0]
    for data in detections.boxes.data.tolist():
        # extract the confidence (i.e., probability) associated with the detection
        confidence = data[4]

        # filter out weak detections by ensuring the 
        # confidence is greater than the minimum confidence
        if float(confidence) < threshold:
            continue

        # if the confidence is greater than the minimum confidence,
        # draw the bounding box on the frame
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        cv.rectangle(frame, (xmin, ymin) , (xmax, ymax), green, 2)

    # end time to compute the fps
    end = datetime.datetime.now()
    # show the time it took to process 1 frame
    total = (end - start).total_seconds()
    # print(f"Time to process 1 frame: {total * 1000:.0f} milliseconds")

    # calculate the frame per second and draw it on the frame
    fps = f"FPS: {1 / total:.2f}"
    cv.putText(frame, fps, (50, 50),
                cv.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

    # show the frame to our screen
    cv.imshow("Frame", frame)
    writer.write(frame)
    if cv.waitKey(1) == ord("q"):
        break

cap.release()
writer.release()
cv.destroyAllWindows()


0: 480x640 3 persons, 67.9ms
Speed: 4.2ms preprocess, 67.9ms inference, 14.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 36.1ms
Speed: 5.9ms preprocess, 36.1ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 36.6ms
Speed: 3.4ms preprocess, 36.6ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 36.4ms
Speed: 3.4ms preprocess, 36.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 37.1ms
Speed: 1.6ms preprocess, 37.1ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 36.4ms
Speed: 1.5ms preprocess, 36.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 36.5ms
Speed: 2.9ms preprocess, 36.5ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 34.9ms
Speed: 1.8ms preprocess, 34.9ms inference, 4.0ms postprocess per image at shape (1, 3, 

In [9]:
cap.release()

In [10]:
cv.destroyAllWindows()